In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from timeit import default_timer as timer

import pytorch_lightning as pl
from pytorch_forecasting import RMSE,MAE,MAPE
from prettytable import PrettyTable

### Loading data
appId userId rating : int32

In [2]:
# 讀入"train/test/val userId/appId/rating" 

train_userId = torch.load('./data/google.train.userId.pt') 
train_appId = torch.load('./data/google.train.appId.pt')
train_rating = torch.load('./data/google.train.rating.pt')

val_userId = torch.load('./data/google.val.userId.pt')  
val_appId = torch.load('./data/google.val.appId.pt') 
val_rating = torch.load('./data/google.val.rating.pt') 

test_userId = torch.load('./data/google.test.userId.pt')  
test_appId = torch.load('./data/google.test.appId.pt')
test_rating = torch.load('./data/google.test.rating.pt') 

In [3]:
# 降維
train_userId = train_userId.view(-1)  
train_appId = train_appId.view(-1)  
train_rating = train_rating.view(-1) 

val_userId = val_userId.view(-1) 
val_appId =val_appId.view(-1)
val_rating = val_rating.view(-1)

test_userId = test_userId.view(-1) 
test_appId = test_appId.view(-1)
test_rating = test_rating.view(-1)

In [4]:
# get_user_item_matrix_indices
user_indices, item_indices, ratings = [], [], []
for idx,i in enumerate(train_rating):
    user_indices.append(train_userId[idx])
    item_indices.append(train_appId[idx])
    ratings.append(i)
    
user_item_rating_indices = [np.array(user_indices), np.array(item_indices), np.array(ratings)]   
user_indices, item_incides, rating_data = user_item_rating_indices

### 模型參數

In [5]:
batch_size = 32
epochs = 50
learning_rate =0.0001
layers = [128,64] #原文設置 # 深度矩陣分解部分的層數與神經元數量
layers_cat1 = [128,64,1] # user和item 串聯後的各dnn層神經元數:128/64


### TensorDataset、Dataloader
Dataloader：把 Dataset類轉換成方便model處理的東西

In [6]:
from torch.utils.data import DataLoader, Dataset
class UserItemRatingDataset(Dataset):
    def __init__(self, user, item, target):
        self.user = user
        self.item = item
        self.target = target
        
    def __getitem__(self, index):
        return self.user[index], self.item[index], self.target[index]
    
    def __len__(self):
        return self.user.size(0)

In [7]:
train_dataset = UserItemRatingDataset(train_userId, train_appId, train_rating)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = UserItemRatingDataset(test_userId, test_appId, test_rating)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

val_dataset = UserItemRatingDataset(val_userId, val_appId, val_rating)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

### 構建神經元網絡

In [8]:
class RA_model(torch.nn.Module):
    def __init__(self, num_users, num_items, layers, layers_cat1):  
        
        super().__init__()  #繼承父class torch.nn.Module
        self.num_users = num_users
        self.num_items = num_items
        self.latent_dim = layers[0]
        self.layers = layers
        self.latent_dim_concat1 = layers_cat1[0] #pi qj串連後再丟進dnn網路訓練
        self.layers_cat1 = layers_cat1
        self.user_str = 'torch.int64'
        #傳入評分矩陣數據
        self.user_item_indices = torch.LongTensor([user_indices, item_indices]) #行列: 即 userId/itemId
        # user_item_indices:存在rating的人所對應的 user/item 組
        self.rating_data = train_rating
        self.user_item_matrix = torch.sparse_coo_tensor(self.user_item_indices,
                                                        self.rating_data,
                                                        torch.Size((self.num_users, self.num_items))).to_dense().to(device)
        '''
        torch.sparse_coo_tensor 存稀疏矩陣
        torch.spares_coo_tensor(indices(行&列), values(評分), siez=None(矩陣大小),*, dtype=None, requires_grad=False)->Tensor
        https://remotedesktop.google.com/access/session/dd38facf-dab1-aca9-3734-98b9f771622
        '''
        # 先分別給定用戶/項目第一層神經網路的參數                                        
        self.linear_user_1 = nn.Linear(in_features=self.num_items, out_features=self.latent_dim)
        self.linear_user_1.weight.detach().normal_(0, 0.01) # 類似手動讓權重初始化的一種方法 
        self.linear_item_1 = nn.Linear(in_features=self.num_users, out_features=self.latent_dim)
        self.linear_item_1.weight.detach().normal_(0, 0.01)  
        
        self.linear_concat1_1 = nn.Linear(in_features=128, out_features=self.latent_dim_concat1) # in_features=use+item
        self.linear_concat1_1.weight.detach().normal_(0, 0.01)  
        '''
        nn.ModuleList() 和 nn.Sequential 一樣是一種容器，可以把任意nn.Module的子類(如nn.Conv2d, nn.Linear)加到該list，
        方法同一般list(extend，append等)    https://zhuanlan.zhihu.com/p/64990232
        '''
        self.user_fc_layers = nn.ModuleList()
        for idx in range(1, len(self.layers)):
            self.user_fc_layers.append(nn.Linear(in_features=self.layers[idx - 1], out_features=self.layers[idx]))

        self.item_fc_layers = nn.ModuleList()
        for idx in range(1, len(self.layers)):
            self.item_fc_layers.append(nn.Linear(in_features=self.layers[idx - 1], out_features=self.layers[idx]))
            
        self.concat1_layers = nn.ModuleList()
        for idx in range(1, len(self.layers_cat1)):
            self.concat1_layers.append(nn.Linear(in_features=self.layers_cat1[idx - 1], out_features=self.layers_cat1[idx]))
        
        self.dropout_layer  = torch.nn.Dropout(0.5)
        
    def forward(self, user_indices, item_indices, idx):
        
        user = self.user_item_matrix[user_indices] # user:稀疏矩陣裡第幾筆評分的用戶是誰 
        item = self.user_item_matrix[:, item_indices].t() # 轉置矩陣  
        
        user = self.linear_user_1(user)
        item = self.linear_item_1(item)

        for idx in range(len(self.layers) - 1):
            user = F.relu(user)
            user = self.dropout_layer(user)
            user = self.user_fc_layers[idx](user)

        for idx in range(len(self.layers) - 1):
            item = F.relu(item)
            item = self.dropout_layer(item)
            item = self.item_fc_layers[idx](item) 

        concat_user_item = torch.cat((user, item), 1)  #串聯pi和qj
        
        #丟入神經網路訓練      
        concat_user_item = self.linear_concat1_1(concat_user_item)
        for idx in range(len(self.layers_cat1) - 1):
            concat_user_item = F.relu(concat_user_item)
            concat_user_item = self.dropout_layer(concat_user_item)
            concat_user_item = self.concat1_layers[idx](concat_user_item)  
        y_hat = concat_user_item.view(-1)
        return y_hat

In [9]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

#build model
ra_model = RA_model(135331,9095, layers, layers_cat1).to(device) # num_users=135331, num_items=9095
optimizer = torch.optim.Adam(ra_model.parameters(),lr=learning_rate)  


### train

In [ ]:
loss_f = torch.nn.MSELoss()
min_val_loss = 10

for epoch in range(epochs):
    train_loss_sum = 0.0
    val_loss_sum = 0.0
    train_loss = 0.0
    val_loss = 0.0 
    start_time = timer()
    
    #train model
    ra_model.train()
    for idx, (user, item, y) in enumerate(train_loader): 
        user = user.long()
        item = item.long()
        user, item, y = user.to(device), item.to(device), y.to(device)
        
        # forward
        y_hat = ra_model(user, item, idx)        
        loss = loss_f(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()              
        train_loss_sum += loss.item()

    train_loss = train_loss_sum / len(train_loader)
    train_time = round(timer() - start_time)

    #val
    ra_model.eval()
    for idx,(user,item,y) in enumerate(val_loader):
        
        user = user.long()
        item = item.long()
        
        user, item, y = user.to(device), item.to(device), y.to(device)
        y_hat = ra_model(user, item,idx)   
        loss = loss_f(y_hat, y) 
        val_loss_sum += loss.item()
        
    val_loss = val_loss_sum / len(val_loader)
    
    # val_loss若小於當前最好的loss就把模型存起來
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        
        #若要存在其他地方檔案位置要改
        model_out_file = './model_ra/ramodel-128-64-128-64-valMSE_{:.4f}-batch_size_{}-lr_{}-epoch_{}.model'.format(
                    val_loss,
                    batch_size,
                    learning_rate,
                    epoch+1)
        torch.save(ra_model.state_dict(), model_out_file)
    
    log = f"[Epoch:{epoch+1}] Train MSE: {train_loss:.4f} Val MSE: {val_loss:.4f} Epoch train time = {train_time:.3f}s "
    print(log)

#### 以下test 若要改跑其他模型或檔案 要更改檔案位置

In [22]:
# load model
ra_model = RA_model(135331,9095, layers, layers_cat1).to(device)
ra_model.load_state_dict(torch.load('./model_ra/ramodel-128-64-128-64-valMSE_1.4588-batch_size_32-lr_0.0001-epoch_13.model'))
ra_model.eval()
loss_f = torch.nn.MSELoss()
test_loss_sum = 0.0

lst=[]
y_pred_np = np.array(lst)
y_true_np = np.array(lst)

for idx, (user, item, y) in enumerate(test_loader): 
    user = user.long()
    item = item.long()
    user, item, y = user.to(device), item.to(device), y.to(device)
    y_hat = ra_model(user, item, idx) 
    test_loss = loss_f(y_hat, y)
    optimizer.zero_grad()
    test_loss.backward()
    optimizer.step()              
    test_loss_sum += test_loss.item()
    
    y_hat = y_hat.cpu().detach().numpy()
    y_np = y.cpu().detach().numpy()

    y_pred_np = np.concatenate((y_pred_np,y_hat))
    y_true_np = np.concatenate((y_true_np,y_np))

test_loss = test_loss_sum / len(test_loader)
log = f"Test MSE loss: {test_loss:.5f}  "
print(log)       

Test MSE loss: 1.75163  


In [23]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_true_np,y_pred_np)
rmse = np.sqrt(mean_squared_error(y_true_np,y_pred_np))

print('MSE:',round(mse,5))
print('RMSE:',round(rmse,4))

#MAE
from sklearn.metrics import mean_absolute_error
def mae_value(y_true_np, y_pred_np):
    mae = mean_absolute_error(y_true_np, y_pred_np)
    return mae
MAE = mae_value(y_true_np,y_pred_np)
print('MAE:',round(MAE,4))

#MAPE
from sklearn.metrics import mean_absolute_percentage_error
MAPE = mean_absolute_percentage_error(y_true_np,y_pred_np)*100
print('MAPE:',round(MAPE,4))


MSE: 1.7511
RMSE: 1.3233
MAE: 0.9855
MAPE: 50.8045
